# Lesson 1: Router Engine

## Setup

In [11]:
from customlib.utils.helper import get_openai_api_key, print_response

OPENAI_API_KEY = get_openai_api_key()

In [12]:
import nest_asyncio

nest_asyncio.apply()

## Load Data

In [13]:
from llama_index.core import SimpleDirectoryReader

# load documents
documents = SimpleDirectoryReader(input_files=["resources/metagpt.pdf"]).load_data()

## Define LLM and Embedding model

In [14]:
from llama_index.core.node_parser import SentenceSplitter

splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents)

In [15]:
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

class ConcreteOpenAI(OpenAI):
    def _prepare_chat_with_tools(self):
        pass

Settings.llm = ConcreteOpenAI(model="gpt-3.5-turbo")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-ada-002")

## Define Summary Index and Vector Index over the Same Data

In [16]:
from llama_index.core import SummaryIndex, VectorStoreIndex

summary_index = SummaryIndex(nodes)
vector_index = VectorStoreIndex(nodes)

## Define Query Engines and Set Metadata

In [17]:
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)
vector_query_engine = vector_index.as_query_engine()

In [18]:
from llama_index.core.tools import QueryEngineTool


summary_tool = QueryEngineTool.from_defaults(
    query_engine=summary_query_engine,
    description=(
        "Useful for summarization questions related to MetaGPT"
    ),
)

vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description=(
        "Useful for retrieving specific context from the MetaGPT paper."
    ),
)

## Define Router Query Engine

In [19]:
from llama_index.core.query_engine.router_query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector


query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=[
        summary_tool,
        vector_tool,
    ],
    verbose=True
)

In [20]:
response = query_engine.query("What is the summary of the document?")
print_response(str(response))

Selecting query engine 0: This choice indicates that the document is useful for summarization questions related to MetaGPT..


The document introduces MetaGPT, a meta-programming framework that enhances multi-agent systems based on Large Language Models (LLMs) by incorporating human-like Standardized Operating Procedures (SOPs) to streamline workflows and improve problem-solving processes. MetaGPT assigns specific roles to agents, facilitates structured communication, and ensures efficient collaboration through a communication protocol. It demonstrates state-of-the-art performance in code generation benchmarks, offers a robust solution for developing LLM-based multi-agent systems, and emphasizes the importance of transparency, accountability, and ethical considerations in artificial intelligence development. The document also outlines the development process using MetaGPT for software projects, showcasing its effectiveness in generating executable code, managing information overload, and addressing challenges in natural language programming.

In [21]:
print(len(response.source_nodes))

34


In [24]:
response = query_engine.query(
    "How do agents share information with other agents?"
)
print_response(str(response))

Selecting query engine 1: This choice is more relevant as it specifically mentions retrieving specific context from the MetaGPT paper, which would likely include information on how agents share information..


Agents share information with other agents by utilizing a shared message pool where they can publish structured messages and subscribe to relevant messages based on their profiles. This shared message pool allows all agents to exchange messages directly, enabling them to access messages from other entities transparently. By storing information in this global message pool, agents can retrieve required information without the need to inquire about other agents and wait for their responses, thus enhancing communication efficiency.

## Let's put everything together

In [23]:
response = query_engine.query("Tell me about the ablation study results?")
print_response(str(response))

Selecting query engine 1: The ablation study results are specific context from the MetaGPT paper, making choice 2 the most relevant..


The ablation study results provide insights into the impact of removing different components or features from a system or model. This analysis helps in understanding the contribution and significance of individual elements towards the overall performance or functionality of the system.